# HSE Продвинутые методы машинного обучения

## Семинар 2

### Полина Полунина
polina.polunina@skolkovotech.ru

# Стекинг, вотинг. Градиентный бустинг


Import packages:

In [101]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
#ml models
import scipy as sp
import sklearn
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
#linear regression
import statsmodels.api as sm
#set style for plots
sns.set_style('darkgrid')
#off the warnings
import warnings
warnings.filterwarnings("ignore")

# Задание для Семинара номер 2:


## Часть 1: Для задачи регрессии:

* подгрузите данные "train.csv" 
* поделите данные на тренировочную и тестовую выборки
* обучите: линейную регрессию, svm, random forest
* используйте предсказания от этих трех моделей для проверки качества ансамбля
* используя предложенные техники ансамблей, проверьте, улучшился ли результат

### Решение

*Подгружаем данные*

In [102]:
cols=['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

In [103]:
data = pd.read_csv('train.csv', index_col=0, usecols=cols)
data.head()

MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
Id                                                                           
1           60     8450            7            5       2003          2003   
2           20     9600            6            8       1976          1976   
3           60    11250            7            5       2001          2002   
4           70     9550            7            5       1915          1970   
5           60    14260            8            5       2000          2000   

    BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  ...  WoodDeckSF  OpenPorchSF  \
Id                                              ...                            
1         150          856       856       854  ...           0           61   
2         284         1262      1262         0  ...         298            0   
3         434          920       920       866  ...           0           42   
4         540          756       961       756  ...           0           35   
5         490         1145      1145      1053  ...         192           84   

    EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  \
Id                                                                             
1               0          0            0         0        0       2    2008   
2               0          0            0         0        0       5    2007   
3               0          0            0         0        0       9    2008   
4             272          0            0         0        0       2    2006   
5               0          0            0         0        0      12    2008   

    SalePrice  
Id             
1      208500  
2      181500  
3      223500  
4      140000  
5      250000  

[5 rows x 31 columns]

*Делим данные на тренировочную и тестовую выборки*

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), data.SalePrice, test_size=0.2, random_state=32)

*Averaging*

Алгоритм: 
* обучаем 3 модели на тренировочной выборке
* делаем 3 предикта на тестовой выборке
* усредняем
* считаем целевую метрику $R^2$


In [142]:
# модель 1: линейная регрессия
from sklearn.linear_model import LinearRegression
# определяем и обучаем модель на тренировочной выборке
linreg = LinearRegression().fit(X_train, y_train)
# предсказываем на тестовой выборке
preds_linreg = linreg.predict(X_test)
# считаем скор на тестовой выборке
linreg.score(X_test, y_test)

0.7638868743142141

In [152]:
# модель 2: random forest
from sklearn.ensemble import RandomForestRegressor
# определяем модель
rf = sklearn.ensemble.RandomForestRegressor()
# обучаем модель
model_rf = rf.fit(X_train, y_train)
# предсказываем
preds_rf = model_rf.predict(X_test)
# считаем скор
model_rf.score(X_test, y_test)

0.7979846147381656

In [153]:
# модель 3: svm
from sklearn.svm import SVR
# определяем модель
svm = SVR(kernel='linear')
# обучаем модель
model_svm = svm.fit(X_train, y_train)
# предсказываем
preds_svm = model_svm.predict(X_test)
# считаем скор
model_svm.score(X_test, y_test)

0.698663801279436

In [154]:
# усредняем три предсказания
averaged_preds = (preds_linreg + preds_rf + preds_svm) / 3

In [155]:
# считаем скор от усреднения
from sklearn.metrics import r2_score
r2_score(y_test, averaged_preds)

0.7878624659897309

In [156]:
# считаем взвешенное среднее
averaged_weighted_preds = 0.2 * preds_linreg + 0.7 * preds_rf + 0.1 * predictions
# считаем скор
r2_score(y_test, averaged_weighted_preds)

0.8057152212819878

*Stacking*

Алгоритм:
* обучаем 3 модели
* делаем предсказания на тренировочной и тестовой выборках
* добавляем предсказания от трех моделей в качестве новых признаков в X_train и X_test
* обучаемся любой новой моделью на дополненном тренировочном сете
* предсказываем на дополненной тестовой выборке
* считаем скор

In [157]:
# предсказания на тренировочной выборке для трех моделей
# модель 1
preds_linreg_train = linreg.predict(X_train)
# модель 2
preds_rf_train = model_rf.predict(X_train)
# модель 3
preds_svm_train = model_svm.predict(X_train)

In [159]:
# добавляем предсказания в качестве новых признаков в тренировочную и тестовую выборки
X_train['linreg'] = preds_linreg_train
X_test['linreg'] = preds_linreg
X_train['rf'] = preds_rf_train
X_test['rf'] = preds_rf
X_train['svm'] = preds_svm_train
X_test['svm'] = preds_svm

In [202]:
# обучаем новую модель на дополненной тренировочной выборке
new_model_rf = rf.fit(X_train, y_train)
# считаем скор
new_model_rf.score(X_test, y_test)

0.7742373629981207

*Blending*

Алгоритм:
 * делим тренировочный сет на две части
 * обучаемся тремя моделями на первой части
 * делаем предсказания для второй части и тестовой выборки
 * добавляем предсказания для второй части и тестовой выборки в эту самую вторую часть и тестовую выборку
 * обучаемся новой моделью на дополненной второй части
 * делаем предсказания для дополненной тестовой выборки
 * считаем скор
 

In [204]:
# убираем добавленные на этапе стекинга признаки
X_train = X_train.drop(['linreg', 'rf', 'svm'], axis=1)
X_test = X_test.drop(['linreg', 'rf', 'svm'], axis=1)

In [205]:
# делим тренировочный сет на две части
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train, y_train, test_size=0.4, random_state=32)

In [206]:
# обучаемся тремя моделями на первой части тренировочного сета
# модель 1
linreg = LinearRegression().fit(X_train_1, y_train_1)
preds_linreg_2 = linreg.predict(X_train_2)
preds_linreg_test = linreg.predict(X_test)
# скор на тесте
linreg.score(X_test, y_test)

0.7549490198212486

In [207]:
# модель 2
rf = sklearn.ensemble.RandomForestRegressor()
model_rf = rf.fit(X_train_1, y_train_1)
preds_rf_2 = model_rf.predict(X_train_2)
preds_rf_test = model_rf.predict(X_test)
# считаем скор на тесте
model_rf.score(X_test, y_test)

0.758292548420813

In [210]:
# модель 3
svm = SVR(kernel='linear')
model_svm = svm.fit(X_train_1, y_train_1)
preds_svm_2 = model_svm.predict(X_train_2)
preds_svm_test = model_svm.predict(X_test)
# считаем скор на тесте
model_svm.score(X_test, y_test)

0.687485402687889

In [211]:
# добавляем признаки во вторую часть тренировочной и тестовую выборки
X_train_2['linreg'] = preds_linreg_2
X_test['linreg'] = preds_linreg_test
X_train_2['rf'] = preds_rf_2
X_test['rf'] = preds_rf_test
X_train_2['svm'] = preds_svm_2
X_test['svm'] = preds_svm_test

In [218]:
# обучаемся новой моделью на дополненной второй части тренировочной выборки
new_model_rf = rf.fit(X_train_2, y_train_2)
# считаем скор
new_model_rf.score(X_test, y_test)

0.8160085251079233

*Bagging*

Алгоритм:
* делим тренировочный датасет на N частей с возвращением (бутстреппинг)
* обучаем на каждой части разные модели и делаем предикт на тестовой выборке
* финальное предсказание - усреднение всех предсказаний
* считаем скор

In [ ]:
# убираем из тестовой выборки добавленные при блендинге фичи
X_test = X_test.drop(['linreg', 'rf', 'svm'], axis=1)

In [247]:
from sklearn.tree import DecisionTreeRegressor

In [306]:
# вспомогательный массив для предсказаний различных моделей
preds = []

# число моделей
n = 40
# кол-во строк в подвыборках
m = 300

# цикл для обучения моделей
for i in range(n):
    # подвыборка тренировочного сета: генерируем индексы строк случайным образом
    idx = np.random.choice(X_train.shape[0], m, replace=False)
    # подвыборка тренировочного сета: находим подвыборку
    X_sample = X_train.iloc[idx, :]
    y_sample = y_train.iloc[idx]
    # определяем модель, обучаем, делаем предсказания
    reg = DecisionTreeRegressor()
    model = reg.fit(X_sample, y_sample)
    predictions = model.predict(X_test)
    # сохраняем предсказания в наш вспомогательный массив
    preds.append(predictions)
    # выводим на экран индивидуальные скоры на тесте
    print('выборка {} | R2 на тесте: {:.2f}'.format(i+1, model.score(X_test, y_test)))
    
# финальное предсказание - усреднение всех предыдущих
final_preds = np.mean(preds, axis=0)
# считаем скор
print('\n\nфинальный скор на тесте: {:.2f}'.format(r2_score(y_test, final_preds)))

выборка 1 | R2 на тесте: 0.70
выборка 2 | R2 на тесте: 0.70
выборка 3 | R2 на тесте: 0.60
выборка 4 | R2 на тесте: 0.65
выборка 5 | R2 на тесте: 0.65
выборка 6 | R2 на тесте: 0.57
выборка 7 | R2 на тесте: 0.52
выборка 8 | R2 на тесте: 0.62
выборка 9 | R2 на тесте: 0.50
выборка 10 | R2 на тесте: 0.67
выборка 11 | R2 на тесте: 0.66
выборка 12 | R2 на тесте: 0.65
выборка 13 | R2 на тесте: 0.69
выборка 14 | R2 на тесте: 0.63
выборка 15 | R2 на тесте: 0.68
выборка 16 | R2 на тесте: 0.67
выборка 17 | R2 на тесте: 0.64
выборка 18 | R2 на тесте: 0.67
выборка 19 | R2 на тесте: 0.65
выборка 20 | R2 на тесте: 0.65
выборка 21 | R2 на тесте: 0.68
выборка 22 | R2 на тесте: 0.68
выборка 23 | R2 на тесте: 0.53
выборка 24 | R2 на тесте: 0.58
выборка 25 | R2 на тесте: 0.55
выборка 26 | R2 на тесте: 0.50
выборка 27 | R2 на тесте: 0.63
выборка 28 | R2 на тесте: 0.59
выборка 29 | R2 на тесте: 0.59
выборка 30 | R2 на тесте: 0.62
выборка 31 | R2 на тесте: 0.64
выборка 32 | R2 на тесте: 0.64
выборка 33 | R2 н

## Часть 2: Градиентный бустинг с пакетом catboost

На этом этапе мы будем использовать полный набор данных, но сначала уберем все признаки с большим кол-вом пропусков в данных, а затем уберем все оставшиеся строки с пропусками

In [309]:
# подгружаем полный набор данных
data = pd.read_csv('train.csv', index_col=0)
data.head()

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
2          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
3          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
4          Lvl    AllPub    Corner  ...        0    NaN   NaN         NaN   
5          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   

   MiscVal MoSold  YrSold  SaleType  SaleCondition  SalePrice  
Id                                                             
1        0      2    2008        WD         Normal     208500  
2        0      5    2007        WD         Normal     181500  
3        0      9    2008        WD         Normal     223500  
4        0      2    2006        WD        Abnorml     140000  
5        0     12    2008        WD         Normal     250000  

[5 rows x 80 columns]

In [310]:
# удаляем признаки с кол-вом пропусков больше 200
mask = (data.isna().sum() <= 200)
cols = data.isna().sum()[mask].index.tolist()
data = data[cols]
# удаляем оставшиеся строки с пропусками 
data.dropna(inplace=True)
cat_features = data.dtypes[data.dtypes == 'object'].index.tolist()

Пакет catboost умеет работать с категориальными признаками, для этого сохраним названия столбцов с категориальными данными в отдельный массив

In [311]:
cat_features = data.dtypes[data.dtypes == 'object'].index.tolist()

Разделим данные на тренировочную, валидационную и тестовую выборки

In [312]:
# делим данные первый раз на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), data.SalePrice, test_size=0.2, random_state=32)
# делим тренировочный кусок на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Импортируем catboost

In [313]:
from catboost import CatBoostRegressor, Pool

Используем Pool конструктор, чтобы создать тренировочный, валидационный и тестовый сеты. Передаем в соответствующую переменную имена категориальных столбцов.

In [323]:
train_data = Pool(X_train, y_train, cat_features=cat_features)
val_data = Pool(X_val, y_val, cat_features=cat_features)
test_data = Pool(X_test, y_test, cat_features=cat_features)

Прописываем желаемые параметры для алгоритма:
* iterations - кол-во итераций
* learning_rate - параметр, используется для варьирования шага градиента; чем значение ниже, тем медленнее сходится алгоритм
* depth - глубина деревьев, оптимальные значения от 4 до 10
* eval_metric - желаемая метрика для мониторинга качества модели
* verbose - параметр, отвечающий за то, выводить ли результаты каждой итерации на экран

Ссылка на описание всех возможных параметров для тюнинга модели: https://catboost.ai/docs/concepts/parameter-tuning.html

Обратите внимание, что learning rate можно формально не прописывать, тогда он будет подобран автоматический в зависимости от кол-ва итераций

In [325]:
params={'iterations' : 500,
        'learning_rate' : 0.03,
        'eval_metric' : 'R2',
        'depth' : 5,
        'verbose' : True
        }

Далее все выглядит привычным нам образом

In [327]:
# определяем модель
model = CatBoostRegressor(**params)

In [328]:
# обучаем модель на тренировочных данных, при этом на каждой итерации мониторим результат на валидационной выборке
model.fit(train_data, eval_set=val_data)

0:	learn: 0.0034681	test: 0.0041094	best: 0.0041094 (0)	total: 70.5ms	remaining: 35.2s
1:	learn: 0.0088492	test: 0.0091889	best: 0.0091889 (1)	total: 140ms	remaining: 34.8s
2:	learn: 0.0109156	test: 0.0124937	best: 0.0124937 (2)	total: 215ms	remaining: 35.7s
3:	learn: 0.0154288	test: 0.0177270	best: 0.0177270 (3)	total: 289ms	remaining: 35.8s
4:	learn: 0.0166346	test: 0.0190894	best: 0.0190894 (4)	total: 326ms	remaining: 32.3s
5:	learn: 0.0214098	test: 0.0234166	best: 0.0234166 (5)	total: 398ms	remaining: 32.7s
6:	learn: 0.0270514	test: 0.0291991	best: 0.0291991 (6)	total: 488ms	remaining: 34.3s
7:	learn: 0.0337481	test: 0.0352915	best: 0.0352915 (7)	total: 586ms	remaining: 36s
8:	learn: 0.0411919	test: 0.0419029	best: 0.0419029 (8)	total: 669ms	remaining: 36.5s
9:	learn: 0.0446383	test: 0.0450045	best: 0.0450045 (9)	total: 752ms	remaining: 36.9s
10:	learn: 0.0500254	test: 0.0505311	best: 0.0505311 (10)	total: 838ms	remaining: 37.2s
11:	learn: 0.0574758	test: 0.0583037	best: 0.0583037 

96:	learn: 0.7927153	test: 0.7225165	best: 0.7225165 (96)	total: 7.65s	remaining: 31.8s
97:	learn: 0.7954766	test: 0.7246201	best: 0.7246201 (97)	total: 7.74s	remaining: 31.7s
98:	learn: 0.7983017	test: 0.7276938	best: 0.7276938 (98)	total: 7.8s	remaining: 31.6s
99:	learn: 0.8006621	test: 0.7298563	best: 0.7298563 (99)	total: 7.89s	remaining: 31.6s
100:	learn: 0.8025283	test: 0.7312756	best: 0.7312756 (100)	total: 7.96s	remaining: 31.4s
101:	learn: 0.8051539	test: 0.7341919	best: 0.7341919 (101)	total: 8.03s	remaining: 31.3s
102:	learn: 0.8069293	test: 0.7357171	best: 0.7357171 (102)	total: 8.08s	remaining: 31.1s
103:	learn: 0.8094840	test: 0.7373863	best: 0.7373863 (103)	total: 8.14s	remaining: 31s
104:	learn: 0.8120701	test: 0.7400989	best: 0.7400989 (104)	total: 8.22s	remaining: 30.9s
105:	learn: 0.8137785	test: 0.7411533	best: 0.7411533 (105)	total: 8.3s	remaining: 30.9s
106:	learn: 0.8156466	test: 0.7428482	best: 0.7428482 (106)	total: 8.36s	remaining: 30.7s
107:	learn: 0.8168828	

191:	learn: 0.8856125	test: 0.7893258	best: 0.7893258 (191)	total: 15.3s	remaining: 24.6s
192:	learn: 0.8861691	test: 0.7890619	best: 0.7893258 (191)	total: 15.4s	remaining: 24.5s
193:	learn: 0.8866220	test: 0.7892890	best: 0.7893258 (191)	total: 15.5s	remaining: 24.5s
194:	learn: 0.8868599	test: 0.7895230	best: 0.7895230 (194)	total: 15.6s	remaining: 24.5s
195:	learn: 0.8870914	test: 0.7891038	best: 0.7895230 (194)	total: 15.7s	remaining: 24.4s
196:	learn: 0.8871620	test: 0.7891560	best: 0.7895230 (194)	total: 15.8s	remaining: 24.3s
197:	learn: 0.8875422	test: 0.7892564	best: 0.7895230 (194)	total: 15.9s	remaining: 24.2s
198:	learn: 0.8881757	test: 0.7895279	best: 0.7895279 (198)	total: 16s	remaining: 24.2s
199:	learn: 0.8885956	test: 0.7896373	best: 0.7896373 (199)	total: 16.1s	remaining: 24.1s
200:	learn: 0.8890590	test: 0.7899794	best: 0.7899794 (200)	total: 16.2s	remaining: 24.1s
201:	learn: 0.8893289	test: 0.7903559	best: 0.7903559 (201)	total: 16.2s	remaining: 24s
202:	learn: 0.

285:	learn: 0.9045486	test: 0.8061785	best: 0.8061785 (285)	total: 22.7s	remaining: 17s
286:	learn: 0.9046920	test: 0.8061899	best: 0.8061899 (286)	total: 22.8s	remaining: 16.9s
287:	learn: 0.9047392	test: 0.8063960	best: 0.8063960 (287)	total: 22.8s	remaining: 16.8s
288:	learn: 0.9048906	test: 0.8063066	best: 0.8063960 (287)	total: 22.9s	remaining: 16.7s
289:	learn: 0.9049193	test: 0.8063604	best: 0.8063960 (287)	total: 23s	remaining: 16.6s
290:	learn: 0.9054565	test: 0.8069080	best: 0.8069080 (290)	total: 23.1s	remaining: 16.6s
291:	learn: 0.9055975	test: 0.8070163	best: 0.8070163 (291)	total: 23.2s	remaining: 16.5s
292:	learn: 0.9057822	test: 0.8070444	best: 0.8070444 (292)	total: 23.3s	remaining: 16.4s
293:	learn: 0.9062426	test: 0.8071547	best: 0.8071547 (293)	total: 23.4s	remaining: 16.4s
294:	learn: 0.9062754	test: 0.8072252	best: 0.8072252 (294)	total: 23.5s	remaining: 16.3s
295:	learn: 0.9064083	test: 0.8076146	best: 0.8076146 (295)	total: 23.6s	remaining: 16.2s
296:	learn: 0.

378:	learn: 0.9160505	test: 0.8170686	best: 0.8171764 (377)	total: 31.7s	remaining: 10.1s
379:	learn: 0.9160975	test: 0.8172718	best: 0.8172718 (379)	total: 31.8s	remaining: 10s
380:	learn: 0.9161363	test: 0.8172647	best: 0.8172718 (379)	total: 31.9s	remaining: 9.96s
381:	learn: 0.9161738	test: 0.8173211	best: 0.8173211 (381)	total: 32s	remaining: 9.88s
382:	learn: 0.9163078	test: 0.8174600	best: 0.8174600 (382)	total: 32.1s	remaining: 9.81s
383:	learn: 0.9163639	test: 0.8176019	best: 0.8176019 (383)	total: 32.2s	remaining: 9.73s
384:	learn: 0.9164141	test: 0.8175594	best: 0.8176019 (383)	total: 32.3s	remaining: 9.65s
385:	learn: 0.9165345	test: 0.8176144	best: 0.8176144 (385)	total: 32.4s	remaining: 9.57s
386:	learn: 0.9166003	test: 0.8176003	best: 0.8176144 (385)	total: 32.5s	remaining: 9.49s
387:	learn: 0.9167226	test: 0.8179736	best: 0.8179736 (387)	total: 32.6s	remaining: 9.41s
388:	learn: 0.9168131	test: 0.8179649	best: 0.8179736 (387)	total: 32.7s	remaining: 9.34s
389:	learn: 0.

470:	learn: 0.9226462	test: 0.8208023	best: 0.8209701 (467)	total: 41.6s	remaining: 2.56s
471:	learn: 0.9227986	test: 0.8211031	best: 0.8211031 (471)	total: 41.7s	remaining: 2.47s
472:	learn: 0.9228107	test: 0.8211918	best: 0.8211918 (472)	total: 41.8s	remaining: 2.39s
473:	learn: 0.9228186	test: 0.8212175	best: 0.8212175 (473)	total: 41.9s	remaining: 2.3s
474:	learn: 0.9228465	test: 0.8211730	best: 0.8212175 (473)	total: 42s	remaining: 2.21s
475:	learn: 0.9230104	test: 0.8213314	best: 0.8213314 (475)	total: 42.2s	remaining: 2.13s
476:	learn: 0.9230316	test: 0.8212814	best: 0.8213314 (475)	total: 42.3s	remaining: 2.04s
477:	learn: 0.9232829	test: 0.8212493	best: 0.8213314 (475)	total: 42.4s	remaining: 1.95s
478:	learn: 0.9237087	test: 0.8216426	best: 0.8216426 (478)	total: 42.5s	remaining: 1.86s
479:	learn: 0.9237183	test: 0.8217068	best: 0.8217068 (479)	total: 42.6s	remaining: 1.77s
480:	learn: 0.9237907	test: 0.8217011	best: 0.8217068 (479)	total: 42.7s	remaining: 1.69s
481:	learn: 0

Что мы видим на экране выше? 

* в первой колонке значение выбранной метрики, в нашей случае - $R^2$, на тренировочной выборке
* во второй колонке - на валидационной выборке
* в третьей колонке мы видим лучший результат на валидации за все обучение и номер соответствующей итерации
* в четвертой колонке общее кол-во потраченного времени на обучение
* в пятой колонке оставшее на обучение время

Что делать дальше?

* Обучаем модель с разными параметрами (пока не надоест) пока не добьемся наилучшего результата на валидационной выборке
* Далее, смотрим на какой итерации был наилучший результат и обучаемся на объединение тренировочной и валидационной выборки с ограничением до этой итерации
* Делаем предикт на тестовой выборке
* Считаем скор (если настоящие значения целевой переменной доступны)

In [330]:
# объединяем тренировочную и валидационную выборки в один сет 
X_train_new = pd.concat([X_train, X_val], axis=0)
y_train_new = pd.concat([y_train, y_val], axis=0)

In [331]:
# используем конструктор Pool
train_data = Pool(X_train_new, y_train_new, cat_features=cat_features)

In [332]:
# задаем параметры модели: те же, что и для наилучшей выбранной модели, но с урезанным кол-вом итерации до наилучшей итерации по выбранной модели
params={'iterations' : 494,
        'learning_rate' : 0.03,
        'eval_metric' : 'R2',
        'depth' : 5,
        'verbose' : True
        }

In [333]:
# определяем и обучаем модель
model = CatBoostRegressor(**params)
model.fit(train_data)

0:	learn: 0.0050129	total: 70.3ms	remaining: 34.6s
1:	learn: 0.0105715	total: 146ms	remaining: 36s
2:	learn: 0.0140408	total: 247ms	remaining: 40.4s
3:	learn: 0.0192710	total: 345ms	remaining: 42.3s
4:	learn: 0.0248677	total: 434ms	remaining: 42.5s
5:	learn: 0.0304701	total: 521ms	remaining: 42.4s
6:	learn: 0.0353528	total: 613ms	remaining: 42.6s
7:	learn: 0.0410780	total: 697ms	remaining: 42.4s
8:	learn: 0.0457823	total: 784ms	remaining: 42.3s
9:	learn: 0.0522430	total: 837ms	remaining: 40.5s
10:	learn: 0.0546811	total: 854ms	remaining: 37.5s
11:	learn: 0.0622411	total: 919ms	remaining: 36.9s
12:	learn: 0.0694814	total: 980ms	remaining: 36.3s
13:	learn: 0.0777871	total: 1.09s	remaining: 37.5s
14:	learn: 0.0847594	total: 1.14s	remaining: 36.4s
15:	learn: 0.0915871	total: 1.21s	remaining: 36.2s
16:	learn: 0.1008455	total: 1.27s	remaining: 35.7s
17:	learn: 0.1090001	total: 1.36s	remaining: 36.1s
18:	learn: 0.1195271	total: 1.44s	remaining: 36.1s
19:	learn: 0.1298374	total: 1.53s	remainin

161:	learn: 0.8709842	total: 13s	remaining: 26.6s
162:	learn: 0.8713460	total: 13.1s	remaining: 26.6s
163:	learn: 0.8716952	total: 13.2s	remaining: 26.6s
164:	learn: 0.8723671	total: 13.4s	remaining: 26.6s
165:	learn: 0.8729496	total: 13.5s	remaining: 26.7s
166:	learn: 0.8735766	total: 13.6s	remaining: 26.7s
167:	learn: 0.8740838	total: 13.7s	remaining: 26.7s
168:	learn: 0.8746198	total: 13.8s	remaining: 26.6s
169:	learn: 0.8748778	total: 13.9s	remaining: 26.5s
170:	learn: 0.8757175	total: 14s	remaining: 26.4s
171:	learn: 0.8760004	total: 14s	remaining: 26.3s
172:	learn: 0.8765987	total: 14.1s	remaining: 26.2s
173:	learn: 0.8775786	total: 14.2s	remaining: 26.1s
174:	learn: 0.8776556	total: 14.3s	remaining: 26.1s
175:	learn: 0.8777793	total: 14.4s	remaining: 25.9s
176:	learn: 0.8782492	total: 14.5s	remaining: 25.9s
177:	learn: 0.8785134	total: 14.5s	remaining: 25.8s
178:	learn: 0.8786956	total: 14.7s	remaining: 25.8s
179:	learn: 0.8792141	total: 14.8s	remaining: 25.8s
180:	learn: 0.8796

320:	learn: 0.9125113	total: 26.4s	remaining: 14.2s
321:	learn: 0.9129603	total: 26.5s	remaining: 14.2s
322:	learn: 0.9129938	total: 26.6s	remaining: 14.1s
323:	learn: 0.9134986	total: 26.7s	remaining: 14s
324:	learn: 0.9135368	total: 26.9s	remaining: 14s
325:	learn: 0.9135742	total: 27s	remaining: 13.9s
326:	learn: 0.9136384	total: 27.1s	remaining: 13.8s
327:	learn: 0.9138051	total: 27.2s	remaining: 13.8s
328:	learn: 0.9138438	total: 27.3s	remaining: 13.7s
329:	learn: 0.9139060	total: 27.4s	remaining: 13.6s
330:	learn: 0.9139147	total: 27.5s	remaining: 13.5s
331:	learn: 0.9144272	total: 27.6s	remaining: 13.5s
332:	learn: 0.9144919	total: 27.7s	remaining: 13.4s
333:	learn: 0.9145754	total: 27.8s	remaining: 13.3s
334:	learn: 0.9146800	total: 28s	remaining: 13.3s
335:	learn: 0.9148556	total: 28s	remaining: 13.2s
336:	learn: 0.9152591	total: 28.1s	remaining: 13.1s
337:	learn: 0.9155093	total: 28.2s	remaining: 13s
338:	learn: 0.9156310	total: 28.3s	remaining: 12.9s
339:	learn: 0.9157349	to

479:	learn: 0.9292881	total: 43.8s	remaining: 1.28s
480:	learn: 0.9293585	total: 43.9s	remaining: 1.19s
481:	learn: 0.9293923	total: 44s	remaining: 1.1s
482:	learn: 0.9294130	total: 44.1s	remaining: 1s
483:	learn: 0.9294303	total: 44.2s	remaining: 914ms
484:	learn: 0.9294551	total: 44.3s	remaining: 823ms
485:	learn: 0.9295158	total: 44.5s	remaining: 732ms
486:	learn: 0.9296446	total: 44.5s	remaining: 640ms
487:	learn: 0.9296967	total: 44.6s	remaining: 549ms
488:	learn: 0.9299939	total: 44.8s	remaining: 458ms
489:	learn: 0.9300385	total: 44.9s	remaining: 366ms
490:	learn: 0.9301270	total: 45s	remaining: 275ms
491:	learn: 0.9301923	total: 45.1s	remaining: 183ms
492:	learn: 0.9302318	total: 45.2s	remaining: 91.7ms
493:	learn: 0.9302574	total: 45.3s	remaining: 0us


In [335]:
# получаем предсказания на тестовых данных
preds = model.predict(test_data)

In [336]:
# считаем скор
r2_score(y_test, preds)

0.882478595648297